<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumSubroutineAlgorithms_QAE_Grok_mkbahk_20250812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-aer==0.15.1
%pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 14.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=2e4fc12b5e97d325d03595b728ec854fad4ddcf074758e04230e2426e89290c4
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 59.2 MB/s eta 0:00:00
  Cloning https://github.com/qiskit-community/qiskit-textbook.git to /tmp/pip-req-build-81o8b2d8
  Running command git clone --filter=blob:none --quiet https://github.com/qiskit-community/qiski

In [2]:
%pip install qiskit-algorithms==0.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 9.3 MB/s eta 0:00:00


In [3]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_state_city, plot_bloch_multivector
from qiskit_textbook.tools import array_to_latex
from qiskit_aer import Aer, AerSimulator
from IPython.display import display, Math
import numpy as np

#from qiskit.primitives import Sampler

In [7]:
from math import asin, sqrt

#from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms import AmplitudeEstimation, EstimationProblem
#from qiskit_algorithms.amplitude_estimators import MaximumLikelihoodAmplitudeEstimation
from qiskit_algorithms import MaximumLikelihoodAmplitudeEstimation

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
# 추정할 확률 정의
p = 0.2

# 베르누이 A 연산자 회로 정의 (상태 준비)
class BernoulliA(QuantumCircuit):
    def __init__(self, probability):
        super().__init__(1)  # 1 큐비트 회로
        theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(theta_p, 0)
    ###def
###class

# 베르누이 Q 연산자 회로 정의 (Grover 연산자, power 메서드 포함)
class BernoulliQ(QuantumCircuit):
    def __init__(self, probability):
        super().__init__(1)  # 1 큐비트 회로
        self._theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(2 * self._theta_p, 0)
    ###def

    def power(self, k):
        q_k = QuantumCircuit(1)
        q_k.ry(2 * k * self._theta_p, 0)
        return q_k
    ###def
###class

# A와 Q 인스턴스 생성
A = BernoulliA(p)
Q = BernoulliQ(p)

# 추정 문제 정의
problem = EstimationProblem(
    state_preparation=A,  # A 연산자
    grover_operator=Q,    # Q 연산자
    objective_qubits=[0], # "좋은" 상태는 큐비트 0에서 |1> 측정
)

# Sampler 초기화 (회로 실행을 위한 프리미티브)
sampler = Sampler()

# Canonical Amplitude Estimation 사용
ae = AmplitudeEstimation(
    num_eval_qubits=3,  # 평가 큐비트 수 (정확도와 회로 너비 결정)
    sampler=sampler,
)

# 진폭 추정 실행
ae_result = ae.estimate(problem)
print("추정된 확률:", ae_result.estimation)

추정된 확률: 0.1464466
